In [51]:
import pandas as pd
import numpy as np
from pymatgen import core
import re

cwd = '/home/graduate/aqd5773/dummy_group_repositiory/'

df = pd.read_csv(cwd+'data/hw5-iron-alloy.csv')
df

,Names,"Elastic (Young's, Tensile) Modulus",Elongation at Break,Poisson's Ratio,Shear Modulus,Tensile Strength: Ultimate (UTS),Tensile Strength: Yield (Proof),Latent Heat of Fusion,Melting Completion (Liquidus),Melting Onset (Solidus),...,Calcium (Ca)Ca,Boron (B)B,Oxygen (O)O,Rockwell B Hardness,Calomel Potential,Tantalum (Ta)Ta,Cerium (Ce)Ce,Lanthanum (La)La,Lead (Pb)Pb,Zinc (Zn)Zn
0,AWS-E100C-K3-or-E69C-K3-Weld-Metal,190,18,0.29,73,770,700,250,1460,1410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AWS-E110C-K3-or-E76C-K3-Weld-Metal,190,17,0.29,73,870,760,250,1460,1410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AWS-E110C-K4-or-E76C-K4-Weld-Metal,190,17,0.29,73,850,780,250,1460,1410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AWS-E120C-K4-or-E83C-K4-Weld-Metal,190,17,0.29,73,950,840,250,1460,1410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AWS-E70C-B2L-or-E49C-B2L-W52130-Weld-Metal,190,21,0.29,73,580,460,260,1460,1420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,UNS-S35045-Alloy-803-Stainless-Steel,200,39,0.28,78,540,190,310,1390,1340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,UNS-S35125-Stainless-Steel,200,39,0.28,78,540,230,300,1430,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,UNS-S35135-Alloy-864-Stainless-Steel,200,34,0.28,79,590,230,320,1430,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,UNS-S35140-Stainless-Steel,200,34,0.28,78,690,310,300,1420,1370,...,NaN,NaN,NaN,88,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
def parse_range(value):
    """Cleans up two problems with this DataFrame:
    (1) NaN values should be 0 for compositions
    (2) Ranges with scalars separated by "to"
    """
    try:
        return float(value)
    except:
        pass
    if type(value) is str and 'to' in value:
        lo, hi = [float(it) for it in value.split('to')]
        return float(0.5 * (lo + hi))

def cleanup_column(column):
    try:
        return column.astype(float)
    except:
        pass
    return [parse_range(it) for it in column.values]

# for column in df.columns:
#     column = cleanup_column(df.iloc[:, 1])

In [53]:

for columns in df.columns:
    try:
        x = columns[columns.find('(')+1:columns.find(')')] #find if column contains elemental info
        if core.Element(x):
            df[columns] = df[columns].fillna(value=0)
    except:
        pass
    if df[columns].dtypes == object:
        try:
            c = cleanup_column(df[columns])
            df[columns] = c
        except:
            pass


In [54]:
df.to_csv(cwd+'data/hw5-iron-alloy-preprocessed.csv')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 73 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Names                                             1302 non-null   object 
 1   Elastic (Young's, Tensile) Modulus                1302 non-null   float64
 2   Elongation at Break                               1100 non-null   float64
 3   Poisson's Ratio                                   1302 non-null   float64
 4   Shear Modulus                                     1302 non-null   float64
 5   Tensile Strength: Ultimate (UTS)                  1299 non-null   float64
 6   Tensile Strength: Yield (Proof)                   1049 non-null   float64
 7   Latent Heat of Fusion                             1302 non-null   int64  
 8   Melting Completion (Liquidus)                     1302 non-null   int64  
 9   Melting Onset (Soli

Remove NaN containing columns from the dataset

In [22]:
to_remove_columns = []
for f in df.columns:
    if (df[f].isna().sum()/len(df))>0.8:
        to_remove_columns.append(f)
        print(f)

Residualsres.
Reduction in Area
Fracture Toughness
Compressive (Crushing) Strength
Curie Temperature
Rockwell C Hardness
Rockwell B Hardness
Calomel Potential


In [29]:
new_df = df.copy()
new_df.drop(columns=to_remove_columns,inplace=True)
new_df.to_csv(cwd+'data/hw5-iron-alloy-preprocessed_v2.csv')

Simple imputation

In [30]:
for f in new_df.columns:
    if new_df[f].isna().sum() != 0 :
        new_df[f].interpolate(inplace=True)

In [36]:
new_df.to_csv(cwd+'data/hw5-iron-alloy-preprocessed_v2_imputed.csv')